In [18]:
from pyspark.sql import SparkSession

In [19]:
spark = SparkSession.builder.appName('trial').getOrCreate()

In [20]:
lines = spark.readStream.format('socket').option('host','127.0.0.1').option('port', 9999).load()

In [21]:
from pyspark.sql.functions import explode,split

In [22]:
words = lines.select(explode(split(lines.value,' ')).alias('word'))

In [23]:
word_tag = words.filter('word LIKE "#%"')

In [24]:
wordCount = word_tag.groupBy('word').count()


In [25]:
wordCount.createOrReplaceTempView('tweets')

In [26]:
query = wordCount.sql_ctx.sql('SELECT * FROM tweets ORDER BY count desc LIMIT 5')\
        .writeStream.format('console').outputMode('complete').start()

In [27]:
query.awaitTermination(30)

False

In [28]:
query.stop()

In [29]:
query.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}